In [ ]:
!pip install Unidecode --quiet
import unidecode 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 7.8 MB/s eta 0:00:00


In [ ]:
import numpy as np 
import re
import nltk
from nltk.corpus import stopwords

# Se cargan los corpus


In [ ]:
# Montamos el drive
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/DEPRESSION/corpus_depression.txt', 'r') as f:
    corpus_depression = f.read()

In [ ]:
# Visualizamos el contenido del corpus de depresion
print(corpus_depression[:408])

I want to enjoy relationships with people but I cant I just dont enjoy other peoples company. Every conversation feels like a burden to me. The shitty thing is, I desparately want to be able to naturally experience relationships with people. But I cant. Every day I try, but its so exhausting, and I never feel any closer or more satisfied I guess you could say in my relationship with whoever Im talking to.


## Separamos por documentos

In [ ]:
def separate_documents(corpus):
    # Inicialización de variables
    documents = []
    documents = corpus.strip().split('\n\n')

    return documents

In [ ]:
doc = separate_documents(corpus_depression)

In [ ]:
doc[3]

'I dont fucking know. Dont ask me. I dont get a say in it. Id like to try to, but nothing I do works, and so why tire myself? For the future? To hopefully feel better? Im not interested in my life anymore. I dont get to be. And Im not going to poison my brain with pharmecuticals which themselves are extremely poorly understood by the scientific community. Im not going to be reliant on drugs just to feel something that might be what normal is supposed to feel like. Drugs that could have averse effects on me.\nI didnt ask to be here. I didnt ask to be born. I didnt come into this life willingly. At least thats what it feels like. '

# Creación del script del dataset

In [ ]:
# Se crea un script de dataset utilizando la librería de Hugging Face
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.8 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
# Se crea una lista de diccionarios con los campos "id" y "text" para cada documento en doc
data = [{"id": str(i), "text": doc[i]} for i in range(len(doc))]

In [ ]:
# Se crea el dataset con la lista de diccionarios, hacemos el split
train_size = int(0.8 * len(data))
val_size = int(0.1 * len(data))
test_size = len(data) - train_size - val_size

train_data = data[:train_size]
val_data = data[train_size:train_size+val_size]
test_data = data[train_size+val_size:]

dataset = DatasetDict({'train': train_data, 'validation': val_data, 'test': test_data})

In [ ]:
# Obtener los conjuntos de entrenamiento del objeto dataset
train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

In [ ]:
import os

In [ ]:
# Rutas completas donde se almacenarán los archivos de texto de los conjuntos
train_file_path = os.path.join('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/DEPRESSION/split', 'train.txt')
val_file_path = os.path.join('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/DEPRESSION/split', 'validation.txt')
test_file_path = os.path.join('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/DEPRESSION/split', 'test.txt')

In [ ]:
# Abrir el archivo de texto para escribir los datos de entrenamiento
with open(train_file_path, 'w', encoding='utf-8') as f:
    for example in train_data:
        # Escribir cada ejemplo de entrenamiento en una línea separada
        f.write(example['text'] + '\n')

# Abrir el archivo de texto para escribir los datos de validacion
with open(val_file_path, 'w', encoding='utf-8') as f:
    for example in val_data:
        # Escribir cada ejemplo de validacion en una línea separada
        f.write(example['text'] + '\n')

# Abrir el archivo de texto para escribir los datos de test
with open(test_file_path, 'w', encoding='utf-8') as f:
    for example in test_data:
        # Escribir cada ejemplo de test en una línea separada
        f.write(example['text'] + '\n')

#Entrenamiento del modelo

In [ ]:
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.1 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/huggingface/transformers.git
!cp transformers/examples/pytorch/language-modeling/run_clm.py /content/transformers/examples/pytorch/language-modeling/

Cloning into 'transformers'...
remote: Enumerating objects: 141772, done.
remote: Counting objects: 100% (1801/1801), done.
remote: Compressing objects: 100% (830/830), done.
remote: Total 141772 (delta 1120), reused 1315 (delta 817), pack-reused 139971
Receiving objects: 100% (141772/141772), 139.78 MiB | 17.97 MiB/s, done.
Resolving deltas: 100% (106047/106047), done.
cp: 'transformers/examples/pytorch/language-modeling/run_clm.py' and '/content/transformers/examples/pytorch/language-modeling/run_clm.py' are the same file


In [ ]:
# Configuración del entrenamiento
model_checkpoint = "gpt2" # Modelo base
output_dir = "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Depression" # Carpeta donde se guardarán los modelos y los logs de entrenamiento
num_train_epochs = 3 # Número de épocas de entrenamiento
learning_rate = 5e-5 # Tasa de aprendizaje
per_device_train_batch_size = 2 # Tamaño del lote de entrenamiento por dispositivo
per_device_eval_batch_size = 2 # Tamaño del lote de evaluación por dispositivo
warmup_steps = 500 # Número de pasos de calentamiento
logging_dir = "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/logs/generative/Depression" # Carpeta donde se guardarán los registros de entrenamiento
logging_steps = 500 # Frecuencia de los registros de entrenamiento
save_steps = 500 # Frecuencia de guardar el modelo

In [ ]:
!pip install -r /content/transformers/examples/pytorch/language-modeling/requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.7 MB/s eta 0:00:00


In [ ]:
!pip install git+https://github.com/huggingface/transformers --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Se instala el módulo "evaluate" que no está instalado en el entorno en el que se está ejecutando el script
!pip install evaluate --quiet

In [ ]:
#%run /content/transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path gpt2 \
    --train_file "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/DEPRESSION/split/train.txt" \
    --validation_file "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/DEPRESSION/split/validation.txt" \
    --do_train \
    --do_eval \
    --output_dir "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Depression" \
    --num_train_epochs 3 \
    --learning_rate 5e-5 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --warmup_steps 500 \
    --logging_dir "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/logs/generative/Depression" \
    --logging_steps 500 \
    --save_steps 500 \
    --fp16 \
    --overwrite_output_dir \
    --report_to tensorboard

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=

INFO:datasets.download.download_manager:Downloading took 0.0 min
INFO:datasets.download.download_manager:Checksum Computation took 0.0 min


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:datasets.builder:Generating train split


Generating train split: 0 examples [00:00, ? examples/s]

...

Step,Training Loss
500,3.508000
1000,3.392700
1500,3.306000
2000,3.262000
2500,3.213400
3000,3.168900
3500,3.145000


[INFO|trainer.py:2904] 2023-05-18 12:43:31,257 >> Saving model checkpoint to /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Depression/checkpoint-500
[INFO|configuration_utils.py:458] 2023-05-18 12:43:31,270 >> Configuration saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Depression/checkpoint-500/config.json
[INFO|configuration_utils.py:364] 2023-05-18 12:43:31,279 >> Configuration saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Depression/checkpoint-500/generation_config.json
[INFO|modeling_utils.py:1836] 2023-05-18 12:43:32,654 >> Model weights saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Depression/checkpoint-500/pytorch_model.bin
[INFO|tokenization_utils_base.py:2181] 2023-05-18 12:43:33,441 >> tokenizer config file saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/outp

***** train metrics *****
  epoch                    =        3.0
  total_flos               =  3656047GF
  train_loss               =     3.2771
  train_runtime            = 0:25:15.39
  train_samples            =       2504
  train_samples_per_second =      4.957
  train_steps_per_second   =      2.479


***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =      0.346
  eval_loss               =     3.2491
  eval_runtime            = 0:00:22.02
  eval_samples            =        313
  eval_samples_per_second =     14.212
  eval_steps_per_second   =      7.128
  perplexity              =    25.7682


[INFO|modelcard.py:451] 2023-05-18 13:05:53,004 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.34603480960277827}]}


# Generamos una muestra de texto a partir de una semilla

In [ ]:
from transformers import pipeline

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
!pip install xformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 MB 9.4 MB/s eta 0:00:00


In [ ]:
model_path = "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Depression/checkpoint-3500"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

[INFO|tokenization_utils_base.py:1808] 2023-05-18 13:06:12,799 >> loading file vocab.json
[INFO|tokenization_utils_base.py:1808] 2023-05-18 13:06:12,803 >> loading file merges.txt
[INFO|tokenization_utils_base.py:1808] 2023-05-18 13:06:12,807 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:1808] 2023-05-18 13:06:12,812 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:1808] 2023-05-18 13:06:12,816 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:1808] 2023-05-18 13:06:12,820 >> loading file tokenizer_config.json
[INFO|configuration_utils.py:667] 2023-05-18 13:06:12,948 >> loading configuration file /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Depression/checkpoint-3500/config.json
[INFO|configuration_utils.py:725] 2023-05-18 13:06:12,952 >> Model config GPT2Config {
  "_name_or_path": "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Depression/checkp

In [ ]:
# Definir la configuración del generador de texto
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [ ]:
# Definir la semilla para la generación de texto
prompt = "Hello, how are you doing?"
# Generar texto a partir de la semilla
text = generator(prompt, max_length=150, do_sample=True, temperature=0.7)
print(text[0]['generated_text'])

[INFO|configuration_utils.py:577] 2023-05-18 13:08:52,294 >> Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.30.0.dev0"
}

[WARNING|utils.py:1282] 2023-05-18 13:08:52,300 >> Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, how are you doing?
Lets just say I am a failure and I dont know what I want Tig is my best friend, but she has the strength to get me through school and I have the strength to try and help her. I am a failure and I just feel like no matter how hard I try I am always wrong. 
amp
I feel like Im just going to never be able to get better with my life and I dont know how to help myself. I dont know what to do. I dont know why I am here. I feel like Im doomed to be this way.
amp
Why am I so frustrated because Ive been in and out of therapy, Ive been diagnosed with ADHD and have
